## 날씨 예측 모델
* 모든 train data 사용
* train shape : (368088, 11)

In [7]:
import pandas as pd
import numpy as np
import time

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor 
from sklearn.ensemble import RandomForestRegressor   # 앙상블(의사결정트리 확장판)
from sklearn.ensemble import GradientBoostingRegressor

import xgboost as xgb
import lightgbm as lgb

In [8]:
train = pd.read_csv("../CSV/train_data.csv", parse_dates=["일시"])

In [9]:
train.head(3)

,연월일,시간,일시,year,month,day,hour,weekday,구분,구분_int,공급량,기온(°C)
0,2013-01-01,1,2013-01-01 00:00:00,2013,1,1,0,1,A,0,2497.129,-8.3
1,2013-01-01,2,2013-01-01 01:00:00,2013,1,1,1,1,A,0,2363.265,-8.5
2,2013-01-01,3,2013-01-01 02:00:00,2013,1,1,2,1,A,0,2258.505,-8.4


In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 368088 entries, 0 to 368087
Data columns (total 12 columns):
 #   Column   Non-Null Count   Dtype         
---  ------   --------------   -----         
 0   연월일      368088 non-null  object        
 1   시간       368088 non-null  int64         
 2   일시       368088 non-null  datetime64[ns]
 3   year     368088 non-null  int64         
 4   month    368088 non-null  int64         
 5   day      368088 non-null  int64         
 6   hour     368088 non-null  int64         
 7   weekday  368088 non-null  int64         
 8   구분       368088 non-null  object        
 9   구분_int   368088 non-null  int64         
 10  공급량      368088 non-null  float64       
 11  기온(°C)   368088 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(7), object(2)
memory usage: 33.7+ MB


In [11]:
train.shape

(368088, 12)

In [12]:
start_time = time.time()

# 2013 ~ 2018
X = train[["year", "month", "day", "hour", "weekday", "구분_int"]]
y = train["기온(°C)"]

model_list = ["LinearRegression", 
              "DecisionTreeRegressor", 
              "RandomForestRegressor", 
              "GradientBoostingRegressor", 
              "xgboost", 
              "lightgbm"]

models = [LinearRegression(), 
         DecisionTreeRegressor(), 
         RandomForestRegressor(n_jobs = -1, random_state = 37), 
         GradientBoostingRegressor(random_state = 37), 
         xgb.XGBRegressor(), 
         lgb.LGBMRegressor()]

test_size = []
train_score = []
test_score = []
MAE = []
MSE = []
RMSE = []
model_name = []
    
for idx, model in enumerate(models) :
    for i in range(1, 10, 1) :
        X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                           test_size = i / 10,
                                                           random_state = 77)
        print("model :", model_list[idx])
        model.fit(X_train, y_train)
        pred = model.predict(X_test)

        # 결정계수 확인
        print("학습용 : {}, 테스트용 : {}".format(10 - i, i))
        print("학습용 데이터 결정계수: {:.3f}".format(model.score(X_train, y_train)))
        print("테스트 데이터 결정계수: {:.3f}".format(model.score(X_test, y_test)))

        # 평가 지표
        print("MAE :", mean_absolute_error(y_test, pred))
        print("MSE :", mean_squared_error(y_test, pred))
        print("RMSE : ", np.sqrt(mean_squared_error(y_test, pred)))
        print()
        test_size.append(i)
        train_score.append(model.score(X_train, y_train))
        test_score.append(model.score(X_test, y_test))
        MAE.append(mean_absolute_error(y_test, pred))
        MSE.append(mean_squared_error(y_test, pred))
        RMSE.append(np.sqrt(mean_squared_error(y_test, pred)))
        model_name.append(model_list[idx])
        
print("실행 시간 : {:.3f}".format(time.time() - start_time))

model : LinearRegression
학습용 : 9, 테스트용 : 1
학습용 데이터 결정계수: 0.069
테스트 데이터 결정계수: 0.071
MAE : 9.305141034096563
MSE : 118.02569759877431
RMSE :  10.863963254667897

model : LinearRegression
학습용 : 8, 테스트용 : 2
학습용 데이터 결정계수: 0.069
테스트 데이터 결정계수: 0.067
MAE : 9.298272040674076
MSE : 117.93302128977273
RMSE :  10.859697108564895

model : LinearRegression
학습용 : 7, 테스트용 : 3
학습용 데이터 결정계수: 0.070
테스트 데이터 결정계수: 0.067
MAE : 9.301098803942113
MSE : 118.0333956346593
RMSE :  10.864317541137101

model : LinearRegression
학습용 : 6, 테스트용 : 4
학습용 데이터 결정계수: 0.070
테스트 데이터 결정계수: 0.068
MAE : 9.305788587478053
MSE : 118.17225532082799
RMSE :  10.870706293559218

model : LinearRegression
학습용 : 5, 테스트용 : 5
학습용 데이터 결정계수: 0.070
테스트 데이터 결정계수: 0.068
MAE : 9.305608903693207
MSE : 118.09094976323249
RMSE :  10.866965987028417

model : LinearRegression
학습용 : 4, 테스트용 : 6
학습용 데이터 결정계수: 0.069
테스트 데이터 결정계수: 0.069
MAE : 9.302286892214498
MSE : 118.00651536465494
RMSE :  10.863080381027057

model : LinearRegression
학습용 : 3, 테스트용 : 

model : lightgbm
학습용 : 3, 테스트용 : 7
학습용 데이터 결정계수: 0.950
테스트 데이터 결정계수: 0.949
MAE : 1.9885516714927514
MSE : 6.443196916383105
RMSE :  2.538345310705993

model : lightgbm
학습용 : 2, 테스트용 : 8
학습용 데이터 결정계수: 0.949
테스트 데이터 결정계수: 0.948
MAE : 2.0042918233446994
MSE : 6.591448558278562
RMSE :  2.5673816541913985

model : lightgbm
학습용 : 1, 테스트용 : 9
학습용 데이터 결정계수: 0.949
테스트 데이터 결정계수: 0.948
MAE : 2.0192983860100675
MSE : 6.664531024988275
RMSE :  2.5815752991125933

실행 시간 : 294.273


In [13]:
print("test_size length : ", len(test_size))
print("train_score length :", len(train_score))
print("test_score length :", len(test_score))
print("MAE length :", len(MAE))
print("MSE length :", len(MSE))
print("RMSE length :", len(RMSE))
print("model_name length :", len(model_name))

test_size length :  54
train_score length : 54
test_score length : 54
MAE length : 54
MSE length : 54
RMSE length : 54
model_name length : 54


In [14]:
data_dict = {"model" : model_name, "test_size" : test_size, "train_score" : train_score, 
            "test_score" : test_score, "MAE" : MAE, "MSE" : MSE, "RMSE" : RMSE}
df = pd.DataFrame(data_dict)

df["train_score"] = df["train_score"] * 100
df["test_score"] = df["test_score"] * 100
df

,model,test_size,train_score,test_score,MAE,MSE,RMSE
0,LinearRegression,1,6.868515,7.110992,9.305141,118.025698,10.863963
1,LinearRegression,2,6.927650,6.745599,9.298272,117.933021,10.859697
2,LinearRegression,3,6.969441,6.708682,9.301099,118.033396,10.864318
3,LinearRegression,4,6.977111,6.759182,9.305789,118.172255,10.870706
4,LinearRegression,5,6.974297,6.805940,9.305609,118.090950,10.866966
5,LinearRegression,6,6.938073,6.856607,9.302287,118.006515,10.863080
6,LinearRegression,7,6.986716,6.845119,9.302820,118.076861,10.866318
7,LinearRegression,8,6.912042,6.883818,9.311450,118.290622,10.876149
8,LinearRegression,9,6.607928,6.907995,9.323260,118.343192,10.878566
9,DecisionTreeRegressor,1,100.000000,99.999810,0.000340,0.000242,0.015541


In [15]:
df.describe()

,test_size,train_score,test_score,MAE,MSE,RMSE
count,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000
mean,5.000000,81.774644,81.651813,2.605166,23.263830,3.219367
std,2.606233,33.934107,33.899754,3.156225,42.981186,3.625313
min,1.000000,6.607928,6.708682,0.000340,0.000242,0.015541
25%,3.000000,91.281592,91.240766,0.275366,0.365454,0.603258
50%,5.000000,96.274055,96.009994,1.728073,5.055580,2.228322
75%,7.000000,99.992975,99.711878,2.588402,11.096402,3.331125
max,9.000000,100.000000,99.999810,9.323260,118.343192,10.878566


In [16]:
df.groupby("model").min()

,test_size,train_score,test_score,MAE,MSE,RMSE
model,,,,,,
DecisionTreeRegressor,1,100.000000,98.307591,0.000340,0.000242,0.015541
GradientBoostingRegressor,1,91.124378,91.186215,2.582127,11.005688,3.317482
LinearRegression,1,6.607928,6.708682,9.298272,117.933021,10.859697
RandomForestRegressor,1,99.854240,99.016629,0.011889,0.000760,0.027572
lightgbm,1,94.873116,94.757497,1.978179,6.393761,2.528589
xgboost,1,97.524669,97.075792,1.351378,3.123557,1.767359


### 최종 모델 선정

In [17]:
# 2013 ~ 2018
X = train[["year", "month", "day", "hour", "weekday", "구분_int"]]
y = train["기온(°C)"]

i = 1

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = i / 10,
                                                    random_state = 77)
model = DecisionTreeRegressor()
model.fit(X_train, y_train)
pred = model.predict(X_test)

# 결정계수 확인
print("학습용 : {}, 테스트용 : {}".format(10 - i, i))
print("학습용 데이터 결정계수: {:.3f}".format(model.score(X_train, y_train)))
print("테스트 데이터 결정계수: {:.3f}".format(model.score(X_test, y_test)))

# 평가 지표
print("MAE :", mean_absolute_error(y_test, pred))
print("MSE :", mean_squared_error(y_test, pred))
print("RMSE : ", np.sqrt(mean_squared_error(y_test, pred)))

학습용 : 9, 테스트용 : 1
학습용 데이터 결정계수: 1.000
테스트 데이터 결정계수: 1.000
MAE : 0.00034230758781858133
MSE : 0.0002417886929826943
RMSE :  0.015549556038121935


### 테스트 데이터 불러오기

In [18]:
test = pd.read_csv("../CSV/new_test.csv")

In [19]:
test.head(3)

,일자|시간|구분,일자,시간,구분,구분_int,일시,year,month,day,hour,weekday
0,2019-01-01 01 A,2019-01-01,1,A,0,2019-01-01 00:00:00,2019,1,1,0,1
1,2019-01-01 02 A,2019-01-01,2,A,0,2019-01-01 01:00:00,2019,1,1,1,1
2,2019-01-01 03 A,2019-01-01,3,A,0,2019-01-01 02:00:00,2019,1,1,2,1


In [20]:
test.tail(3)

,일자|시간|구분,일자,시간,구분,구분_int,일시,year,month,day,hour,weekday
15117,2019-03-31 22 H,2019-03-31,22,H,6,2019-03-31 21:00:00,2019,3,31,21,6
15118,2019-03-31 23 H,2019-03-31,23,H,6,2019-03-31 22:00:00,2019,3,31,22,6
15119,2019-03-31 24 H,2019-03-31,24,H,6,2019-03-31 23:00:00,2019,3,31,23,6


In [21]:
test_x = test[["year", "month", "day", "hour", "weekday", "구분_int"]]

### 19년도 기온 예측

In [22]:
pred = model.predict(test_x)
test["기온"] = pred

In [23]:
test.tail(50)

,일자|시간|구분,일자,시간,구분,구분_int,일시,year,month,day,hour,weekday,기온
15070,2019-03-29 23 H,2019-03-29,23,H,6,2019-03-29 22:00:00,2019,3,29,22,4,12.7
15071,2019-03-29 24 H,2019-03-29,24,H,6,2019-03-29 23:00:00,2019,3,29,23,4,12.1
15072,2019-03-30 01 H,2019-03-30,1,H,6,2019-03-30 00:00:00,2019,3,30,0,5,11.8
15073,2019-03-30 02 H,2019-03-30,2,H,6,2019-03-30 01:00:00,2019,3,30,1,5,11.4
15074,2019-03-30 03 H,2019-03-30,3,H,6,2019-03-30 02:00:00,2019,3,30,2,5,11.0
15075,2019-03-30 04 H,2019-03-30,4,H,6,2019-03-30 03:00:00,2019,3,30,3,5,10.3
15076,2019-03-30 05 H,2019-03-30,5,H,6,2019-03-30 04:00:00,2019,3,30,4,5,9.9
15077,2019-03-30 06 H,2019-03-30,6,H,6,2019-03-30 05:00:00,2019,3,30,5,5,10.4
15078,2019-03-30 07 H,2019-03-30,7,H,6,2019-03-30 06:00:00,2019,3,30,6,5,10.3
15079,2019-03-30 08 H,2019-03-30,8,H,6,2019-03-30 07:00:00,2019,3,30,7,5,10.5


In [24]:
train.head(50)

,연월일,시간,일시,year,month,day,hour,weekday,구분,구분_int,공급량,기온(°C)
0,2013-01-01,1,2013-01-01 00:00:00,2013,1,1,0,1,A,0,2497.129,-8.3
1,2013-01-01,2,2013-01-01 01:00:00,2013,1,1,1,1,A,0,2363.265,-8.5
2,2013-01-01,3,2013-01-01 02:00:00,2013,1,1,2,1,A,0,2258.505,-8.4
3,2013-01-01,4,2013-01-01 03:00:00,2013,1,1,3,1,A,0,2243.969,-8.1
4,2013-01-01,5,2013-01-01 04:00:00,2013,1,1,4,1,A,0,2344.105,-8.2
5,2013-01-01,6,2013-01-01 05:00:00,2013,1,1,5,1,A,0,2390.961,-8.2
6,2013-01-01,7,2013-01-01 06:00:00,2013,1,1,6,1,A,0,2378.457,-8.6
7,2013-01-01,8,2013-01-01 07:00:00,2013,1,1,7,1,A,0,2518.921,-8.3
8,2013-01-01,9,2013-01-01 08:00:00,2013,1,1,8,1,A,0,2706.481,-7.9
9,2013-01-01,10,2013-01-01 09:00:00,2013,1,1,9,1,A,0,2832.057,-7.0


### 예측한 데이터 저장
* test 데이터 프레임 저장

In [ ]:
# test.to_csv("../CSV/pred_test.csv", index = False)

In [27]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize = (15, 10))
sns.lineplot(x = "일자", y = "기온", data = test)

In [ ]:
a = train[(train["year"] == 2013) & ((train["month"] == 1) | (train["month"] == 2) | (train["month"] == 3))]

In [ ]:
plt.figure(figsize = (15, 10))
sns.lineplot(x = "일시", y = "기온(°C)", data = a)

In [ ]:
sns.lineplot(x = "month", y = "기온", data = test)
sns.lineplot(x = "month", y = "기온(°C)", data = a)

In [ ]:
sns.lineplot(x = "month", y = "기온(°C)", hue = "year", data = train)